In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image

In [9]:
class CarDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_json(csv_file, lines=True)
        self.image_dir = image_dir
        self.transform = transform

        # One-Hot Encoding dla zmiennych kategorycznych
        self.data = pd.get_dummies(self.data, columns=['fuel_type', 'gearbox', 'model', 'car_type_main'], drop_first=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Ładowanie obrazu
        img_name = f"{self.image_dir}/{self.data.iloc[idx]['img_local']}"
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)

        # Pobranie danych tabelarycznych (bez kolumny price i img_local)
        tabular_data = self.data.drop(columns=['price', 'img_local']).iloc[idx].values.astype(float)
        
        # Pobranie ceny jako celu
        price = self.data.iloc[idx]['price']
        
        return {'image': image, 'tabular': torch.tensor(tabular_data, dtype=torch.float32), 'price': torch.tensor(price, dtype=torch.float32)}


In [15]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = CarDataset(csv_file='..\\..\\train_data_with_car_type.json', image_dir='..\\..\\data_img', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [16]:
class CarPricePredictor(nn.Module):
    def __init__(self):
        super(CarPricePredictor, self).__init__()
        
        # Ekstraktor cech obrazu - ResNet (bez warstwy końcowej klasyfikacji)
        self.cnn = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Usuwamy ostatnią warstwę FC
        self.cnn_out_features = 512 

        # Sieć MLP dla danych tabelarycznych
        self.tabular_mlp = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # Połączenie wyjść CNN i MLP
        self.fc = nn.Sequential(
            nn.Linear(self.cnn_out_features + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # Wyjście przewidujące cenę
        )

    def forward(self, image, tabular):
        # Przepuszczenie obrazu przez CNN
        cnn_features = self.cnn(image)
        cnn_features = cnn_features.view(cnn_features.size(0), -1)

        # Przepuszczenie danych tabelarycznych przez MLP
        tabular_features = self.tabular_mlp(tabular)

        # Połączenie cech z CNN i MLP
        combined_features = torch.cat((cnn_features, tabular_features), dim=1)

        # Predykcja ceny
        output = self.fc(combined_features)
        return output


In [17]:
# Zakodowanie zmiennych kategorycznych i obliczenie liczby cech tabelarycznych
data = pd.read_json('..\\..\\train_data_with_car_type.json', lines=True)
data_encoded = pd.get_dummies(data, columns=['fuel_type', 'gearbox', 'model', 'car_type_main'], drop_first=True)
tabular_input_dim = data_encoded.drop(columns=['price', 'img_local']).shape[1]
print("Wymiar wejściowy dla danych tabelarycznych:", tabular_input_dim)

Wymiar wejściowy dla danych tabelarycznych: 263


In [18]:
import torch.optim as optim

# Inicjalizacja modelu, optymalizatora i funkcji straty
model = CarPricePredictor(tabular_input_dim=tabular_input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trening
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in dataloader:
        images = batch['image']
        tabular_data = batch['tabular']
        prices = batch['price']

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images, tabular_data).squeeze()
        loss = criterion(outputs, prices)

        # Backward pass i optymalizacja
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")


c:\studia\wejherowo_tree\wejherowo\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\studia\wejherowo_tree\wejherowo\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/25], Loss: 18316222206.0723


KeyboardInterrupt: 

In [7]:
torch.save(model.state_dict(), 'car_price_predictor25n.pth')